<a href="https://colab.research.google.com/github/emaudealtema/Machine_Learning_repo/blob/master/Theme5_NLP_%C3%A9t%C3%A9_2022_classification_des_tweets_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#https://www.kaggle.com/competitions/nlp-getting-started/data?select=train.csv  
# Dans ce travail nous allons utiliser le jeu de donnees Disaster Tweets ou nous allons classifier en 2 categorie de tweets, desaster ou non desaster tweets
# nous utilisons la partie d<entrainement du dataset car les données sont déjà toute étiquées et contient 7613 lignes et 5 colonnes
df = pd.read_csv("/content/drive/MyDrive/theme5-sujets-speciaux-2022-UQAR/train.csv")
df.shape

(7613, 5)

In [ ]:
df.head(20)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [ ]:
# les données avec l'étiquète 1 c'est une donnée desastreuse et 0 pas desastreuse
print((df.target == 1).sum()) # Disaster
print((df.target == 0).sum()) # No Disaster

3271
4342


In [ ]:
import re
import string
# Ici on définit une fonction qui va suprimer les urls dans les tweets
def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

# https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate/34294022
#On définit une autre fonction pour suprimer les ponctuations dans les tweets
def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
#Ici on cherche si il y a des urls dans nos données
pattern = re.compile(r"https?://(\S+|www)\.\S+")
for t in df.text:
    matches = pattern.findall(t)
    for match in matches:
        print(t)
        print(match)
        print(pattern.sub(r"", t))
    if len(matches) > 0:
        break

Arsonist sets cars ablaze at dealership https://t.co/gOQvyJbpVI
t
Arsonist sets cars ablaze at dealership 


In [ ]:
# On fait appel à nos deux fonction pour suprimer les urls et les ponctuations.

df["text"] = df.text.map(remove_URL) # map(lambda x: remove_URL(x))
df["text"] = df.text.map(remove_punct)

In [ ]:
# effacer les mots non utils avec stopword

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Mots vides : un mot vide est un mot couramment utilisé (tel que "le", "un", "une", "dans") qu'un moteur de recherche
# a été programmé pour ignorer, à la fois lors de l'indexation des entrées pour la recherche et lors de leur récupération
# comme résultat d'une requête de recherche.
stop = set(stopwords.words("english"))

# https://stackoverflow.com/questions/5486337/how-to-remove-stop-words-using-nltk-or-python
def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:
df["text"] = df.text.map(remove_stopwords)

In [ ]:
df.text

0           deeds reason #earthquake may allah forgive us
1                  forest fire near la ronge sask. canada
2       residents asked 'shelter place' notified offic...
3       13,000 people receive #wildfires evacuation or...
4       got sent photo ruby #alaska smoke #wildfires p...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    @aria_ahrary @thetawniest control wild fires c...
7610    m1.94 [01:04 utc]?5km volcano hawaii. http://t...
7611    police investigating e-bike collided car littl...
7612    latest: homes razed northern california wildfi...
Name: text, Length: 7613, dtype: object

In [ ]:
# Ici nous comptons les mots uniques puis les met dans un dictionnaire 
from collections import Counter

# Compter les mots uniques
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter = counter_word(df.text)

In [ ]:
len(counter)

27833

In [ ]:
counter

Counter({'deeds': 1,
         'reason': 20,
         '#earthquake': 19,
         'may': 87,
         'allah': 4,
         'forgive': 2,
         'us': 99,
         'forest': 54,
         'fire': 209,
         'near': 54,
         'la': 19,
         'ronge': 1,
         'sask.': 1,
         'canada': 6,
         'residents': 7,
         'asked': 9,
         "'shelter": 1,
         "place'": 1,
         'notified': 1,
         'officers.': 1,
         'evacuation': 47,
         'shelter': 5,
         'place': 22,
         'orders': 11,
         'expected': 12,
         '13,000': 1,
         'people': 174,
         'receive': 2,
         '#wildfires': 5,
         'california': 91,
         'got': 111,
         'sent': 13,
         'photo': 13,
         'ruby': 1,
         '#alaska': 2,
         'smoke': 44,
         'pours': 1,
         'school': 63,
         '#rockyfire': 4,
         'update': 14,
         '=>': 1,
         'hwy.': 4,
         '20': 13,
         'closed': 17,
         'd

In [ ]:
# Juste pour voir les 5 mots les plus fréquent dans nos données et combien de fois ils apparaissent
counter.most_common(5)

[('-', 763), ('like', 341), ('&amp;', 295), ("i'm", 237), ('get', 227)]

In [ ]:
num_unique_words = len(counter)

In [ ]:
# diviser le dataset en donnees d'entrainement et  donnees de  validation
train_size = int(df.shape[0] * 0.8)

train_df = df[:train_size]
val_df = df[train_size:]

# split text and labels
train_sentences = train_df.text.to_numpy()
train_labels = train_df.target.to_numpy()
val_sentences = val_df.text.to_numpy()
val_labels = val_df.target.to_numpy()

In [ ]:
train_sentences.shape, val_sentences.shape

((6090,), (1523,))

In [ ]:

from tensorflow.keras.preprocessing.text import Tokenizer

# vectoriser un corpus de texte en transformant chaque texte en une séquence d'entiers
tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences) # fit only to training

In [ ]:
# chaque mot a index unique 
word_index = tokenizer.word_index
word_index

{'t': 1,
 'co': 2,
 'http': 3,
 'https': 4,
 'amp': 5,
 'like': 6,
 'fire': 7,
 'get': 8,
 '2': 9,
 'via': 10,
 "i'm": 11,
 'new': 12,
 'people': 13,
 'news': 14,
 'emergency': 15,
 'one': 16,
 "'": 17,
 'video': 18,
 'disaster': 19,
 'body': 20,
 'burning': 21,
 'would': 22,
 'buildings': 23,
 'police': 24,
 'u': 25,
 'us': 26,
 '3': 27,
 'day': 28,
 'crash': 29,
 'first': 30,
 'man': 31,
 'still': 32,
 '1': 33,
 'got': 34,
 'know': 35,
 'california': 36,
 'two': 37,
 'back': 38,
 'time': 39,
 'going': 40,
 'full': 41,
 'accident': 42,
 '4': 43,
 'world': 44,
 'attack': 45,
 'see': 46,
 'nuclear': 47,
 'gt': 48,
 'love': 49,
 'rt': 50,
 'youtube': 51,
 'may': 52,
 'year': 53,
 'go': 54,
 'many': 55,
 'watch': 56,
 'collapse': 57,
 'dead': 58,
 'today': 59,
 '5': 60,
 'mass': 61,
 'car': 62,
 '2015': 63,
 'life': 64,
 'want': 65,
 'hiroshima': 66,
 'years': 67,
 'work': 68,
 'train': 69,
 'last': 70,
 'fires': 71,
 'best': 72,
 'good': 73,
 'think': 74,
 'families': 75,
 'way': 76,
 'w

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

In [ ]:
#@title Chaque tweet est représenté par une séquence de mot { display-mode: "code" }
#variable_name = 86 #@param {type:"slider", min:0, max:100, step:1}
print(train_sentences[10:15])
print(train_sequences[10:15])

['three people died heat wave far'
 'haha south tampa getting flooded hah- wait second live south tampa gonna gonna fvck #flooding'
 "#raining #flooding #florida #tampabay #tampa 18 19 days. i've lost count"
 '#flood bago myanmar #we arrived bago'
 'damage school bus 80 multi car crash #breaking']
[[566, 13, 432, 149, 304, 455], [799, 472, 2371, 160, 2372, 2969, 567, 698, 191, 472, 2371, 213, 213, 5924, 133], [2970, 133, 1988, 5925, 2371, 850, 1730, 539, 305, 658, 2971], [107, 3920, 659, 2373, 1528, 3920], [114, 100, 367, 3921, 2374, 62, 29, 325]]


In [ ]:
# Remplir les séquences pour qu'elles aient la même longueur, disons qu'on veut que la longueur max soit 20
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Nombre maximum de mots dans une séquence
max_length = 20
# On le fait pour le train, la validation et le
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding="post", truncating="post")
train_padded.shape, val_padded.shape

((6090, 20), (1523, 20))

In [ ]:
train_padded[10]

array([566,  13, 432, 149, 304, 455,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [ ]:
print(train_sentences[10])
print(train_sequences[10])
print(train_padded[10])

three people died heat wave far
[566, 13, 432, 149, 304, 455]
[566  13 432 149 304 455   0   0   0   0   0   0   0   0   0   0   0   0
   0   0]


In [ ]:
#Ici on fait l'inversion de l<index et de la valeur, fait que lindex va devenir valeur et valeur l'index comme la valeur apparaissait avant l''index dans l'affichage precedente, 
#On cree un dictionnaire pour les recevoir

# remplace clé par  valeur et vice-versa
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])

In [ ]:
#Maintenant nos indexes sont des cléés et doivent apparaitrent avant la valeur qui est le texte pas la valeur avant comme dans l'afficahge précédent
reverse_word_index

{1: 't',
 2: 'co',
 3: 'http',
 4: 'https',
 5: 'amp',
 6: 'like',
 7: 'fire',
 8: 'get',
 9: '2',
 10: 'via',
 11: "i'm",
 12: 'new',
 13: 'people',
 14: 'news',
 15: 'emergency',
 16: 'one',
 17: "'",
 18: 'video',
 19: 'disaster',
 20: 'body',
 21: 'burning',
 22: 'would',
 23: 'buildings',
 24: 'police',
 25: 'u',
 26: 'us',
 27: '3',
 28: 'day',
 29: 'crash',
 30: 'first',
 31: 'man',
 32: 'still',
 33: '1',
 34: 'got',
 35: 'know',
 36: 'california',
 37: 'two',
 38: 'back',
 39: 'time',
 40: 'going',
 41: 'full',
 42: 'accident',
 43: '4',
 44: 'world',
 45: 'attack',
 46: 'see',
 47: 'nuclear',
 48: 'gt',
 49: 'love',
 50: 'rt',
 51: 'youtube',
 52: 'may',
 53: 'year',
 54: 'go',
 55: 'many',
 56: 'watch',
 57: 'collapse',
 58: 'dead',
 59: 'today',
 60: '5',
 61: 'mass',
 62: 'car',
 63: '2015',
 64: 'life',
 65: 'want',
 66: 'hiroshima',
 67: 'years',
 68: 'work',
 69: 'train',
 70: 'last',
 71: 'fires',
 72: 'best',
 73: 'good',
 74: 'think',
 75: 'families',
 76: 'way',
 77

In [ ]:
# Ici on définit une fonction pour decoder les données de la partie d'entrainement, 
#cette fonction doit parcourir toutes la séquence pour trouver l'index de chque mot
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

In [ ]:
# Là on fait appel à notre fonction decode et on lui passe un exemple de données de la partie entrainement puis on affiche le résultat pour voir ce que ça donne
decoded_text = decode(train_sequences[10])

print(train_sequences[10])
print(decoded_text)

[566, 13, 432, 149, 304, 455]
three people died heat wave far


##### On voit comment chaque mot est représenté dans la mémoire de la machine pour que la machine puisse lire les mots et quand on decode on voit chaque mot de la phrase ce que nous on comprend

In [ ]:
# Create LSTM model
from tensorflow.keras import layers

# Embedding: https://www.tensorflow.org/tutorials/text/word_embeddings
# Transforme les entiers positifs (index) en vecteurs denses de taille fixe. (une autre approche pourrait être un encodage à chaud)

# Les incorporations de mots nous permettent d'utiliser une représentation efficace et dense dans laquelle des mots similaires ont
# un encodage similaire. Il est important de noter que vous n'avez pas à spécifier cet encodage à la main. Un encastrement est un
# vecteur dense de valeurs à virgule flottante (la longueur du vecteur est un paramètre que vous spécifiez).

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))

# La couche prendra en entrée une matrice entière de taille (batch, input_length),
# et le plus grand nombre entier (c'est-à-dire l'index des mots) dans l'entrée ne doivent pas être supérieurs à num_mots (taille du vocabulaire).
# Maintenant, model.output_shape est (None, input_length, 32), où `None` est la dimension du lot.


model.add(layers.LSTM(64, dropout=0.1))
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 32)            890656    
                                                                 
 lstm (LSTM)                 (None, 64)                24832     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 915,553
Trainable params: 915,553
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# le crossentropy binaire pour la perte vue qu'on utilise une classification binaire dans notre modèle 
# on passe False à from_logic car on a déjà utilisé la fonction dÈactivation
loss = keras.losses.BinaryCrossentropy(from_logits=False)
# On tuiliser Adam optimizer
optim = keras.optimizers.Adam(lr=0.001)
# On veut Accuracy comme métriqur
metrics = ["accuracy"]
# Puis on compile le modèle
model.compile(loss=loss, optimizer=optim, metrics=metrics)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# On fit le modèle avec train_paddes, train_labels,et on fine tune avec les données de validation, nbr_epoch = 20 
model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=2)

Epoch 1/20
191/191 - 8s - loss: 0.5368 - accuracy: 0.7200 - val_loss: 0.4411 - val_accuracy: 0.8017 - 8s/epoch - 44ms/step
Epoch 2/20
191/191 - 6s - loss: 0.2764 - accuracy: 0.9010 - val_loss: 0.5647 - val_accuracy: 0.7761 - 6s/epoch - 31ms/step
Epoch 3/20
191/191 - 6s - loss: 0.1454 - accuracy: 0.9555 - val_loss: 0.6513 - val_accuracy: 0.7663 - 6s/epoch - 31ms/step
Epoch 4/20
191/191 - 6s - loss: 0.0846 - accuracy: 0.9736 - val_loss: 0.6172 - val_accuracy: 0.7754 - 6s/epoch - 29ms/step
Epoch 5/20
191/191 - 5s - loss: 0.0586 - accuracy: 0.9831 - val_loss: 0.9155 - val_accuracy: 0.7577 - 5s/epoch - 25ms/step
Epoch 6/20
191/191 - 6s - loss: 0.0442 - accuracy: 0.9880 - val_loss: 0.9132 - val_accuracy: 0.7630 - 6s/epoch - 30ms/step
Epoch 7/20
191/191 - 6s - loss: 0.0316 - accuracy: 0.9923 - val_loss: 0.9105 - val_accuracy: 0.7636 - 6s/epoch - 30ms/step
Epoch 8/20
191/191 - 6s - loss: 0.0304 - accuracy: 0.9923 - val_loss: 0.8366 - val_accuracy: 0.7603 - 6s/epoch - 34ms/step
Epoch 9/20
191/1

In [ ]:
# Faisons des prédiction, on fait appel a la fonction prédict
predictions = model.predict(train_padded)
predictions = [1 if p > 0.5 else 0 for p in predictions]

In [ ]:

#Affichons les résultats
print(train_sentences[10:20])

print(train_labels[10:20])
print(predictions[10:20])

['three people died heat wave far'
 'haha south tampa getting flooded hah- wait second live south tampa gonna gonna fvck #flooding'
 "#raining #flooding #florida #tampabay #tampa 18 19 days. i've lost count"
 '#flood bago myanmar #we arrived bago'
 'damage school bus 80 multi car crash #breaking' "what's man?"
 'love fruits' 'summer lovely' 'car fast' 'goooooooaaaaaal!!!!!!']
[1 1 1 1 1 0 0 0 0 0]
[1, 1, 1, 1, 1, 0, 0, 0, 0, 0]


In [ ]:
#En conclusion